In [4]:
import json
from datetime import datetime, timedelta
from itertools import dropwhile, groupby
import numpy as np
import pandas as pd
import scipy.signal as sp
from pathlib import Path
from typing import Mapping, Optional
import re

# Custom functions
from scripts import stat_wind
import plot
import func_vsz as fv


In [11]:
import statsmodels.api as sm  # .stats.api as sms

# Plotting libraries and settings

import matplotlib.pyplot as plt
import matplotlib
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable
from matplotlib.ticker import LogLocator, AutoMinorLocator, MultipleLocator
from matplotlib.gridspec import GridSpec, GridSpecFromSubplotSpec

matplotlib.rcParams["axes.linewidth"] = 1.5
matplotlib.rcParams["figure.figsize"] = (16, 7)
matplotlib.rcParams["font.size"] = 12
matplotlib.rcParams["axes.xmargin"] = 0.001  # (default: 0.05)
matplotlib.rcParams["axes.ymargin"] = 0.01

from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.axes_grid1.axes_size import Fixed, Scaled

try:
    matplotlib.use(
        "Qt5Agg"
    )  # must be before importing plt (raises error after although documentation said no effect)
except ImportError:
    pass
matplotlib.interactive(True)
plt.ion()
# import seaborn as sns
# sns.set_theme(style="whitegrid")


In [14]:
path_raw_adv = Path(
    r"B:\WorkData\Cruises(older)\_KaraSea\220906_AMK89-1\ADV_Sontek\_raw,to_txt,vsz"
)
path_db = Path(
    r"B:\WorkData\Cruises(older)\_KaraSea\220906_AMK89-1\ADV_Sontek\st7440A,7441A@ADV+TCM.h5"
)

adv_str = "ADV"
tcm_str = "TCM"

str_abs, str_dir = "|V|", "Vdir"
n_params = 2  # number of (above) parameters to plot

In [6]:
# Load from DB
print(f"Loading from {path_db}...", end="")
dfs = {}  # stations dict with values of all devices data merged into one dataframe
with pd.HDFStore(path_db, mode="r") as store:
    nodes = sorted(store.root.__members__)  # , key=number_key
    print(nodes)
    st_prefix = "st"
    stations = [node[len(st_prefix):] for node in nodes if node.startswith(st_prefix)]
    # dfs that is needed for regression plots
    for st in stations:
        dfs[st] = store[f"st{st}"].rename(
            lambda col: col.replace("Vabs", "|V|"), axis="columns"
        )

Loading from B:\WorkData\Cruises(older)\_KaraSea\220906_AMK89-1\ADV_Sontek\st7440A,7441A@ADV+TCM.h5...['PSD_i37_orig', 'PSD_i38_orig', 'PSD_i_b21_orig', 'PSD_i_b22_orig', 'st7440A', 'st7441A']


In [ ]:
print("Plotting")
b_plot_source = True  # False
if b_plot_source:
    ## 3.1 Plot source on stations
    for st, df in dfs.items():
        # # Convert dates to days for color points and label colorbar
        # days_start = df.index[0].floor("d")
        # df["days"] = (df.index - days_start).total_seconds() / 86400

        cols_prm = [c for c in df.columns if c.startswith(str_abs)]
        len_str_prm_prefix = len(str_abs) + 1
        devs = [
            c[len_str_prm_prefix:] for c in cols_prm
            if c[len_str_prm_prefix:].startswith("ADV4", "TCM5")
        ]
        devs.sort(
            key=lambda c: (
                lambda b_tcm: (not b_tcm) * 10000  # > max depth to sort TCM first
                + int(c[len(tcm_str if tcm_str else adv_str) : -1])  # sort depth
            )(c.startswith(tcm_str))
        )
        print(st, devs)
        n_rows = len(devs) * n_params  #  = df.columns.size if no new cols

        fig, axes = plt.subplots(nrows=n_rows, sharex=True)
        for i_dev, dev in enumerate(devs):
            for i_prm, (str_prm, str_unit) in enumerate(
                [(str_abs, "{m}/{s}".format_map(fv.I)), (str_dir, "°")]
            ):
                i_plot = i_dev * n_params + i_prm
                ax = axes[i_plot]
                ax.plot(
                    df.index,
                    df[f"{str_prm}_{dev}"],
                    label=dev,
                    color="red" if str_prm == str_abs else "blue",
                )
                if i_plot == n_rows - 1:
                    ax.set_xlabel("Time")
                ax.set_ylabel(", ".join([str_prm, str_unit]))
                ax.grid(True, linestyle="--")
                ax.legend(title=st if i_plot == 0 else None, loc="upper right")

                # cols_prm = [c for c in df.columns if c.startswith(str_prm)]
                # len_str_prm_prefix = len(str_prm) + 1
                # devs = [c[len_str_prm_prefix:] for c in cols_prm]

                # icol_x = devs.index([dev for dev in devs if dev.startswith(tcm_str)][0])
                # # ADV devices
                # col_prm_y = cols_prm[icol_x]

    plt.show()
    print("source data plotted")

Plotting
7440A ['TCM50m', 'ADV44m', 'ADV70m']
7441A ['TCM50m', 'ADV45m', 'ADV76m']
source data plotted


In [ ]:
# All devices data statistics
b_save_stat = False
for st in stations:
    df_stats, df_stats_corr = stat_wind.get_stat(dfs[st], v="V")
    if b_save_stat:
        stat_wind.save_stat(
            df_stats,
            df_stats_corr,
            path_base=path_raw_adv.with_name(f"st{st}_avg-dec={int(dt)}s"),
            v="V",
        )
